# **Chapter 42: Security Best Practices**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Securely store sensitive data (API keys, tokens, credentials) using platform-specific secure storage (Keychain/Keystore)
- Implement certificate pinning and SSL/TLS validation to prevent man-in-the-middle attacks
- Obfuscate Dart code and configure R8/ProGuard to prevent reverse engineering
- Detect rooted/jailbroken devices and implement runtime application self-protection (RASP)
- Mitigate OWASP Mobile Top 10 vulnerabilities (insecure data storage, weak cryptography, insecure communication)
- Implement secure authentication flows with proper token management and biometric authentication
- Configure network security policies and certificate transparency
- Audit dependencies for known vulnerabilities using automated tools

---

## **Prerequisites**

- Completed Chapter 41: Accessibility (understanding of platform channels and native integration)
- Proficiency in platform-specific development (Kotlin/Java for Android, Swift/Objective-C for iOS)
- Understanding of cryptography basics (encryption, hashing, digital signatures)
- Familiarity with OWASP Mobile Security Risks
- Knowledge of HTTP/HTTPS protocols and TLS/SSL handshake
- Access to Android Keystore and iOS Keychain documentation

---

## **42.1 Secure Data Storage**

Mobile applications handle sensitive data (authentication tokens, PII, financial data) that must be protected at rest. Flutter provides platform channels to access native secure storage mechanisms.

### **Platform-Specific Secure Storage**

```dart
// lib/services/secure_storage.dart
import 'dart:convert';
import 'package:flutter/services.dart';
import 'package:crypto/crypto.dart';

/// Exception thrown when secure storage operations fail
class SecureStorageException implements Exception {
  final String code;
  final String message;
  final dynamic details;

  SecureStorageException({
    required this.code,
    required this.message,
    this.details,
  });

  @override
  String toString() => 'SecureStorageException[$code]: $message';
}

/// Service for storing sensitive data securely using platform keychains
/// 
/// Android: Uses Android Keystore System with AES-256 encryption
/// iOS: Uses Keychain Services with kSecAttrAccessibleAfterFirstUnlock
class SecureStorageService {
  static const MethodChannel _channel = 
      MethodChannel('com.example.app/secure_storage');
  
  /// Key derivation using PBKDF2 for additional security layer
  static String _deriveKey(String input, String salt) {
    final bytes = utf8.encode(input + salt);
    final digest = sha256.convert(bytes);
    return digest.toString();
  }

  /// Store sensitive value with platform-level encryption
  /// 
  /// [key] is the identifier for the stored value
  /// [value] is the sensitive data to encrypt and store
  /// [accessGroup] (iOS only) specifies the Keychain access group for shared access
  static Future<void> write({
    required String key,
    required String value,
    String? accessGroup,
    StorageAccessibility accessibility = StorageAccessibility.unlocked,
  }) async {
    try {
      // Input validation
      if (key.isEmpty) {
        throw SecureStorageException(
          code: 'INVALID_KEY',
          message: 'Storage key cannot be empty',
        );
      }

      // Additional entropy for key derivation (optional enhancement)
      final derivedKey = _deriveKey(key, 'app_specific_salt_v1');

      final Map<String, dynamic> arguments = {
        'key': derivedKey,
        'value': value,
        'accessGroup': accessGroup,
        'accessibility': accessibility.name,
      };

      await _channel.invokeMethod('write', arguments);
    } on PlatformException catch (e) {
      throw SecureStorageException(
        code: e.code,
        message: e.message ?? 'Unknown platform error',
        details: e.details,
      );
    }
  }

  /// Retrieve sensitive value from secure storage
  static Future<String?> read({
    required String key,
    String? accessGroup,
  }) async {
    try {
      final derivedKey = _deriveKey(key, 'app_specific_salt_v1');
      
      final result = await _channel.invokeMethod<String>('read', {
        'key': derivedKey,
        'accessGroup': accessGroup,
      });
      
      return result;
    } on PlatformException catch (e) {
      throw SecureStorageException(
        code: e.code,
        message: e.message ?? 'Failed to read from secure storage',
        details: e.details,
      );
    }
  }

  /// Delete specific key from secure storage
  static Future<void> delete({
    required String key,
    String? accessGroup,
  }) async {
    try {
      final derivedKey = _deriveKey(key, 'app_specific_salt_v1');
      
      await _channel.invokeMethod('delete', {
        'key': derivedKey,
        'accessGroup': accessGroup,
      });
    } on PlatformException catch (e) {
      throw SecureStorageException(
        code: e.code,
        message: e.message ?? 'Failed to delete from secure storage',
      );
    }
  }

  /// Clear all stored values (use with caution)
  static Future<void> deleteAll() async {
    try {
      await _channel.invokeMethod('deleteAll');
    } on PlatformException catch (e) {
      throw SecureStorageException(
        code: e.code,
        message: e.message ?? 'Failed to clear secure storage',
      );
    }
  }
}

/// iOS Keychain accessibility levels
/// Determines when the data is accessible to the app
enum StorageAccessibility {
  /// Data accessible only when device is unlocked
  /// Recommended for most sensitive data
  unlocked,
  
  /// Data accessible after first unlock, even if device is subsequently locked
  /// Suitable for background operations
  unlocked_this_device,
  
  /// Data accessible only when device is unlocked and requires passcode
  /// Highest security, but requires device passcode
  password_unlocked,
  
  /// Data accessible regardless of lock state (least secure)
  /// Not recommended for sensitive data
  always,
}
```

**Explanation:**

- **MethodChannel**: Communicates with native platform code to access Android Keystore and iOS Keychain. These APIs are not available directly in Dart.
- **Key Derivation**: Uses SHA-256 hashing with salt to derive storage keys. This adds an additional layer of security even if the keychain is compromised.
- **Accessibility Levels**: iOS Keychain allows specifying when data is accessible (only when unlocked, after first unlock, etc.). `unlocked` ensures data is only accessible when device is unlocked.
- **Access Groups**: iOS supports sharing keychain items between apps in the same team using access groups.
- **Input Validation**: Never trust input parameters; validate keys and values before passing to platform code.
- **Exception Handling**: Convert platform-specific errors into domain-specific exceptions for consistent error handling across platforms.

### **Android Keystore Implementation**

```kotlin
// android/src/main/kotlin/com/example/app/SecureStoragePlugin.kt
package com.example.app

import android.content.Context
import android.content.SharedPreferences
import android.security.keystore.KeyGenParameterSpec
import android.security.keystore.KeyProperties
import android.util.Base64
import androidx.annotation.NonNull
import io.flutter.embedding.engine.plugins.FlutterPlugin
import io.flutter.plugin.common.MethodCall
import io.flutter.plugin.common.MethodChannel
import java.security.KeyStore
import javax.crypto.Cipher
import javax.crypto.KeyGenerator
import javax.crypto.SecretKey
import javax.crypto.spec.GCMParameterSpec

class SecureStoragePlugin : FlutterPlugin, MethodChannel.MethodCallHandler {
    private lateinit var channel: MethodChannel
    private lateinit var context: Context
    
    // Android Keystore provider
    private val androidKeyStore = "AndroidKeyStore"
    private val keyAlias = "app_secure_storage_key"
    private val androidPrefKey = "flutter_secure_storage"
    
    // AES-GCM configuration
    private val transformation = "AES/GCM/NoPadding"
    private val gcmTagLength = 128
    private val gcmIvLength = 12 // 96 bits for GCM

    override fun onAttachedToEngine(@NonNull flutterPluginBinding: FlutterPlugin.FlutterPluginBinding) {
        channel = MethodChannel(flutterPluginBinding.binaryMessenger, "com.example.app/secure_storage")
        channel.setMethodCallHandler(this)
        context = flutterPluginBinding.applicationContext
    }

    override fun onMethodCall(call: MethodCall, result: MethodChannel.Result) {
        when (call.method) {
            "write" -> handleWrite(call, result)
            "read" -> handleRead(call, result)
            "delete" -> handleDelete(call, result)
            "deleteAll" -> handleDeleteAll(result)
            else -> result.notImplemented()
        }
    }

    private fun handleWrite(call: MethodCall, result: MethodChannel.Result) {
        try {
            val key = call.argument<String>("key") ?: throw IllegalArgumentException("Key is required")
            val value = call.argument<String>("value") ?: throw IllegalArgumentException("Value is required")
            
            // Generate or retrieve key from Android Keystore
            val secretKey = getOrCreateSecretKey()
            
            // Encrypt value
            val cipher = Cipher.getInstance(transformation)
            cipher.init(Cipher.ENCRYPT_MODE, secretKey)
            
            val iv = cipher.iv
            val encrypted = cipher.doFinal(value.toByteArray(Charsets.UTF_8))
            
            // Combine IV + encrypted data
            val combined = ByteArray(iv.size + encrypted.size)
            System.arraycopy(iv, 0, combined, 0, iv.size)
            System.arraycopy(encrypted, 0, combined, iv.size, encrypted.size)
            
            // Store in encrypted SharedPreferences
            val encryptedBase64 = Base64.encodeToString(combined, Base64.DEFAULT)
            getSecurePreferences().edit().putString(key, encryptedBase64).apply()
            
            result.success(null)
        } catch (e: Exception) {
            result.error("WRITE_ERROR", e.message, e.stackTraceToString())
        }
    }

    private fun handleRead(call: MethodCall, result: MethodChannel.Result) {
        try {
            val key = call.argument<String>("key") ?: throw IllegalArgumentException("Key is required")
            
            val encryptedBase64 = getSecurePreferences().getString(key, null)
            if (encryptedBase64 == null) {
                result.success(null)
                return
            }
            
            val combined = Base64.decode(encryptedBase64, Base64.DEFAULT)
            
            // Extract IV and ciphertext
            val iv = combined.copyOfRange(0, gcmIvLength)
            val encrypted = combined.copyOfRange(gcmIvLength, combined.size)
            
            // Decrypt
            val secretKey = getOrCreateSecretKey()
            val cipher = Cipher.getInstance(transformation)
            cipher.init(Cipher.DECRYPT_MODE, secretKey, GCMParameterSpec(gcmTagLength, iv))
            
            val decrypted = cipher.doFinal(encrypted)
            val value = String(decrypted, Charsets.UTF_8)
            
            result.success(value)
        } catch (e: Exception) {
            result.error("READ_ERROR", e.message, e.stackTraceToString())
        }
    }

    private fun getOrCreateSecretKey(): SecretKey {
        val keyStore = KeyStore.getInstance(androidKeyStore)
        keyStore.load(null)
        
        // Check if key exists
        if (keyStore.containsAlias(keyAlias)) {
            return keyStore.getEntry(keyAlias, null) as javax.crypto.KeyStore.SecretKeyEntry
        }
        
        // Generate new AES-256 key
        val keyGenerator = KeyGenerator.getInstance(KeyProperties.KEY_ALGORITHM_AES, androidKeyStore)
        val keyGenParameterSpec = KeyGenParameterSpec.Builder(
            keyAlias,
            KeyProperties.PURPOSE_ENCRYPT or KeyProperties.PURPOSE_DECRYPT
        )
            .setBlockModes(KeyProperties.BLOCK_MODE_GCM)
            .setEncryptionPaddings(KeyProperties.ENCRYPTION_PADDING_NONE)
            .setKeySize(256)
            .setUserAuthenticationRequired(false) // Set to true for biometric auth
            .setRandomizedEncryptionRequired(true)
            .build()
        
        keyGenerator.init(keyGenParameterSpec)
        return keyGenerator.generateKey()
    }

    private fun getSecurePreferences(): SharedPreferences {
        return context.getSharedPreferences(androidPrefKey, Context.MODE_PRIVATE)
    }

    private fun handleDelete(call: MethodCall, result: MethodChannel.Result) {
        val key = call.argument<String>("key")
        if (key != null) {
            getSecurePreferences().edit().remove(key).apply()
        }
        result.success(null)
    }

    private fun handleDeleteAll(result: MethodChannel.Result) {
        getSecurePreferences().edit().clear().apply()
        result.success(null)
    }

    override fun onDetachedFromEngine(@NonNull binding: FlutterPlugin.FlutterPluginBinding) {
        channel.setMethodCallHandler(null)
    }
}
```

**Explanation:**

- **Android Keystore**: Hardware-backed keystore (TEE - Trusted Execution Environment) stores cryptographic keys. Keys never leave the secure hardware.
- **AES-256-GCM**: Industry standard for authenticated encryption. GCM provides both confidentiality and integrity (tamper detection).
- **IV Management**: Initialization Vector (IV) is randomly generated for each encryption and prepended to ciphertext. Required for AES-GCM security.
- **SharedPreferences**: Stores encrypted data (not the key). Even if rooted, attacker gets encrypted blobs without the key stored in Keystore.
- **KeyGenParameterSpec**: Configures key generation with security requirements (256-bit, GCM mode, randomized encryption).
- **User Authentication**: Setting `setUserAuthenticationRequired(true)` binds key to biometric/device credential (Android 6.0+).

### **iOS Keychain Implementation**

```swift
// ios/Classes/SecureStoragePlugin.swift
import Flutter
import UIKit
import Security

public class SecureStoragePlugin: NSObject, FlutterPlugin {
    private let serviceName = "com.example.app.secure_storage"
    private let biometricUsageDescription = "Access secure data"
    
    public static func register(with registrar: FlutterPluginRegistrar) {
        let channel = FlutterMethodChannel(
            name: "com.example.app/secure_storage",
            binaryMessenger: registrar.messenger()
        )
        let instance = SecureStoragePlugin()
        registrar.addMethodCallDelegate(instance, channel: channel)
    }
    
    public func handle(_ call: FlutterMethodCall, result: @escaping FlutterResult) {
        switch call.method {
        case "write":
            handleWrite(call: call, result: result)
        case "read":
            handleRead(call: call, result: result)
        case "delete":
            handleDelete(call: call, result: result)
        case "deleteAll":
            handleDeleteAll(result: result)
        default:
            result(FlutterMethodNotImplemented)
        }
    }
    
    private func handleWrite(call: FlutterMethodCall, result: @escaping FlutterResult) {
        guard let args = call.arguments as? [String: Any],
              let key = args["key"] as? String,
              let value = args["value"] as? String else {
            result(FlutterError(code: "INVALID_ARGUMENTS", message: "Key and value required", details: nil))
            return
        }
        
        let accessibility = args["accessibility"] as? String ?? "unlocked"
        let accessGroup = args["accessGroup"] as? String
        
        // Delete existing item first (update pattern)
        deleteItem(key: key, accessGroup: accessGroup)
        
        // Map accessibility string to Keychain constant
        let accessibilityConstant = mapAccessibility(accessibility)
        
        var query: [String: Any] = [
            kSecClass as String: kSecClassGenericPassword,
            kSecAttrAccount as String: key,
            kSecValueData as String: value.data(using: .utf8)!,
            kSecAttrService as String: serviceName,
            kSecAttrAccessible as String: accessibilityConstant
        ]
        
        if let accessGroup = accessGroup {
            query[kSecAttrAccessGroup as String] = accessGroup
        }
        
        let status = SecItemAdd(query as CFDictionary, nil)
        
        if status == errSecSuccess {
            result(nil)
        } else {
            let message = SecCopyErrorMessageString(status, nil) as String? ?? "Unknown error"
            result(FlutterError(code: "WRITE_ERROR", message: message, details: nil))
        }
    }
    
    private func handleRead(call: FlutterMethodCall, result: @escaping FlutterResult) {
        guard let args = call.arguments as? [String: Any],
              let key = args["key"] as? String else {
            result(FlutterError(code: "INVALID_ARGUMENTS", message: "Key required", details: nil))
            return
        }
        
        let accessGroup = args["accessGroup"] as? String
        
        var query: [String: Any] = [
            kSecClass as String: kSecClassGenericPassword,
            kSecAttrAccount as String: key,
            kSecAttrService as String: serviceName,
            kSecReturnData as String: true,
            kSecMatchLimit as String: kSecMatchLimitOne
        ]
        
        if let accessGroup = accessGroup {
            query[kSecAttrAccessGroup as String] = accessGroup
        }
        
        var resultData: AnyObject?
        let status = SecItemCopyMatching(query as CFDictionary, &resultData)
        
        if status == errSecSuccess {
            if let data = resultData as? Data,
               let value = String(data: data, encoding: .utf8) {
                result(value)
            } else {
                result(FlutterError(code: "DECODE_ERROR", message: "Failed to decode data", details: nil))
            }
        } else if status == errSecItemNotFound {
            result(nil) // Key not found is not an error
        } else {
            let message = SecCopyErrorMessageString(status, nil) as String? ?? "Unknown error"
            result(FlutterError(code: "READ_ERROR", message: message, details: nil))
        }
    }
    
    private func handleDelete(call: FlutterMethodCall, result: @escaping FlutterResult) {
        guard let args = call.arguments as? [String: Any],
              let key = args["key"] as? String else {
            result(FlutterError(code: "INVALID_ARGUMENTS", message: "Key required", details: nil))
            return
        }
        
        let accessGroup = args["accessGroup"] as? String
        deleteItem(key: key, accessGroup: accessGroup)
        result(nil)
    }
    
    private func handleDeleteAll(result: @escaping FlutterResult) {
        var query: [String: Any] = [
            kSecClass as String: kSecClassGenericPassword,
            kSecAttrService as String: serviceName
        ]
        
        SecItemDelete(query as CFDictionary)
        result(nil)
    }
    
    private func deleteItem(key: String, accessGroup: String?) {
        var query: [String: Any] = [
            kSecClass as String: kSecClassGenericPassword,
            kSecAttrAccount as String: key,
            kSecAttrService as String: serviceName
        ]
        
        if let accessGroup = accessGroup {
            query[kSecAttrAccessGroup as String] = accessGroup
        }
        
        SecItemDelete(query as CFDictionary)
    }
    
    private func mapAccessibility(_ accessibility: String) -> CFString {
        switch accessibility {
        case "unlocked":
            return kSecAttrAccessibleWhenUnlockedThisDeviceOnly
        case "unlocked_this_device":
            return kSecAttrAccessibleAfterFirstUnlockThisDeviceOnly
        case "password_unlocked":
            return kSecAttrAccessibleWhenPasscodeSetThisDeviceOnly
        case "always":
            return kSecAttrAccessibleAlwaysThisDeviceOnly
        default:
            return kSecAttrAccessibleWhenUnlockedThisDeviceOnly
        }
    }
}
```

**Explanation:**

- **Keychain Services**: iOS secure storage API. Data encrypted with device passcode and hardware keys.
- **kSecClassGenericPassword**: Stores data as generic password items (most flexible for key-value storage).
- **Accessibility Constants**: Determine when data is accessible:
  - `kSecAttrAccessibleWhenUnlockedThisDeviceOnly`: Only when device unlocked, not backed up to iCloud.
  - `kSecAttrAccessibleAfterFirstUnlockThisDeviceOnly`: After first unlock until device restart.
  - `kSecAttrAccessibleWhenPasscodeSetThisDeviceOnly`: Only if device has passcode set.
- **Access Groups**: Allows sharing keychain items between apps in same team (App Group).
- **ThisDeviceOnly**: Prevents data from being included in iCloud backups, keeping it only on device.

---

## **42.2 Network Security**

Protecting data in transit is as critical as protecting data at rest. Implement certificate pinning and secure network configurations.

### **Certificate Pinning**

```dart
// lib/services/security_context.dart
import 'dart:convert';
import 'dart:io';
import 'package:flutter/services.dart';
import 'package:http/io_client.dart';

/// Service for configuring secure HTTP clients with certificate pinning
class SecureHttpClient {
  static SecurityContext? _securityContext;
  
  /// Initialize with embedded certificates
  static Future<HttpClient> createSecureClient() async {
    final context = SecurityContext(withTrustedRoots: false);
    
    // Load Certificate Authority (CA) certificates
    final caCert = await rootBundle.loadString('assets/certs/ca-cert.pem');
    context.setTrustedCertificatesBytes(
      utf8.encode(caCert),
    );
    
    // Load specific server certificate for pinning (optional)
    // This pins to specific certificate, not just CA
    try {
      final serverCert = await rootBundle.loadString('assets/certs/server-cert.pem');
      context.useCertificateChainBytes(utf8.encode(serverCert));
    } catch (e) {
      // Server cert optional, CA pinning is minimum
    }
    
    final httpClient = HttpClient(context: context);
    
    // Enforce TLS 1.2 minimum
    httpClient.badCertificateCallback = (X509Certificate cert, String host, int port) {
      // Implement additional pinning checks here
      _validateCertificate(cert, host);
      return false; // Reject by default, validate manually
    };
    
    return httpClient;
  }
  
  static void _validateCertificate(X509Certificate cert, String host) {
    // Extract SHA-256 fingerprint of certificate
    // Compare against pinned fingerprints
    final fingerprint = _computeFingerprint(cert);
    
    const pinnedFingerprints = {
      'api.example.com': 'AA:BB:CC:DD:EE:FF:...',
      'api-backup.example.com': '11:22:33:44:55:66:...',
    };
    
    final expected = pinnedFingerprints[host];
    if (expected == null) {
      throw CertificatePinningException('Unknown host: $host');
    }
    
    if (fingerprint != expected) {
      throw CertificatePinningException(
        'Certificate pinning failed for $host. '
        'Expected: $expected, Got: $fingerprint'
      );
    }
  }
  
  static String _computeFingerprint(X509Certificate cert) {
    // Implementation would extract public key and compute SHA-256
    // Placeholder for actual implementation
    return cert.sha1Fingerprint ?? 'unknown';
  }
}

class CertificatePinningException implements Exception {
  final String message;
  CertificatePinningException(this.message);
  
  @override
  String toString() => 'CertificatePinningException: $message';
}

// Usage with Dio
class PinnedDioClient {
  static Future<Dio> create() async {
    final httpClient = await SecureHttpClient.createSecureClient();
    final ioClient = IOClient(httpClient);
    
    final dio = Dio();
    dio.httpClientAdapter = IOHttpClientAdapter(
      createHttpClient: () => httpClient,
    );
    
    return dio;
  }
}
```

**Explanation:**

- **SecurityContext**: Dart's API for TLS/SSL configuration. `withTrustedRoots: false` starts with empty trust store (certificate pinning mode).
- **setTrustedCertificatesBytes**: Loads your CA certificates. Only certificates signed by these CAs are trusted (removes trust in system CAs, preventing rogue CA attacks).
- **badCertificateCallback**: Called when certificate validation fails. Return `true` to accept anyway (dangerous), `false` to reject. Use for custom validation logic.
- **Certificate Pinning**: Hardcode expected certificate fingerprints (SHA-256 of public key). MITM attacks using different certificates will fail fingerprint check.
- **TLS 1.2 Enforcement**: Modern security standard. Prevents downgrade attacks to SSLv3 or TLS 1.0/1.1.

### **Android Network Security Config**

```xml
<!-- android/app/src/main/res/xml/network_security_config.xml -->
<?xml version="1.0" encoding="utf-8"?>
<network-security-config>
    <!-- Pin specific certificates -->
    <domain-config>
        <domain includeSubdomains="true">api.example.com</domain>
        
        <!-- Trust anchor for specific certificate -->
        <trust-anchors>
            <certificates src="@raw/cert"/>
        </trust-anchors>
        
        <!-- Pin specific certificate chain -->
        <pin-set expiration="2025-01-01">
            <pin digest="SHA-256">sha256/AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA=</pin>
            <pin digest="SHA-256">sha256/BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB=</pin>
            <!-- Backup pin for certificate rotation -->
        </pin-set>
        
        <!-- Require certificate transparency -->
        <trust-anchors>
            <certificates src="system"/>
        </trust-anchors>
        <certificate-transparency>
            <domain-config>
                <domain includeSubdomains="true">api.example.com</domain>
            </domain-config>
        </certificate-transparency>
    </domain-config>
    
    <!-- Debug configuration (remove for production) -->
    <debug-overrides>
        <trust-anchors>
            <certificates src="user"/>
        </trust-anchors>
    </debug-overrides>
</network-security-config>
```

```xml
<!-- AndroidManifest.xml -->
<application
    android:networkSecurityConfig="@xml/network_security_config"
    ... >
</application>
```

**Explanation:**

- **Network Security Config**: Android 7.0+ declarative configuration for TLS/SSL.
- **Certificate Pinning**: `<pin-set>` defines expected public key hashes. App rejects connections presenting different certificates.
- **Expiration**: Forces app update before certificate expires (prevents outages).
- **Backup Pins**: Include hash of next certificate for rotation without app update downtime.
- **Certificate Transparency**: Monitors public certificate logs for unauthorized certificates issued for your domain.
- **Debug Overrides**: Allows user certificates (Charles Proxy, etc.) only in debug builds. **Critical**: Remove `debug-overrides` for production.

---

## **42.3 Code Obfuscation and Anti-Tampering**

Prevent reverse engineering and tampering with code obfuscation and root detection.

### **Dart Code Obfuscation**

```bash
# Build with obfuscation
flutter build apk --obfuscate --split-debug-info=symbols/
flutter build ios --obfuscate --split-debug-info=symbols/

# symbols/ directory contains mapping files for deobfuscation
# Store securely for crash reporting
```

```yaml
# android/app/proguard-rules.pro
# ProGuard rules for additional obfuscation

# Keep Flutter classes
-keep class io.flutter.app.** { *; }
-keep class io.flutter.plugin.** { *; }
-keep class io.flutter.util.** { *; }
-keep class io.flutter.view.** { *; }
-keep class io.flutter.** { *; }
-keep class io.flutter.plugins.** { *; }

# Keep custom models (if using JSON serialization)
-keep class com.example.app.model.** { *; }

# Keep native methods
-keepclasseswithmembernames class * {
    native <methods>;
}

# Obfuscate everything else
-repackageclasses
-allowaccessmodification

# Remove logging in release
-assumenosideeffects class android.util.Log {
    public static *** d(...);
    public static *** v(...);
    public static *** i(...);
}
```

**Explanation:**

- **Obfuscate**: Renames classes, methods, and variables to meaningless names (a, b, c), making reverse engineering difficult.
- **Split Debug Info**: Separates symbol tables from binary. Required for deobfuscating stack traces from crash reports.
- **ProGuard/R8**: Android's code shrinker and obfuscator. Removes unused code and renames classes.
- **Keep Rules**: Preserve Flutter framework classes and reflection-used classes. Obfuscating these breaks the engine.
- **Security**: Obfuscation is not encryption; it slows down reverse engineering but doesn't prevent it. Combine with other measures.

### **Root and Jailbreak Detection**

```dart
// lib/security/root_detection.dart
import 'dart:io';
import 'package:flutter/services.dart';

/// Runtime Application Self-Protection (RASP) service
class SecurityChecker {
  static const MethodChannel _channel = 
      MethodChannel('com.example.app/security');
  
  /// Check if device is compromised
  static Future<SecurityStatus> checkDeviceSecurity() async {
    final List<SecurityRisk> risks = [];
    
    // Check 1: Root/ Jailbreak detection
    if (await _isDeviceRooted()) {
      risks.add(SecurityRisk.rootedDevice);
    }
    
    // Check 2: Emulator detection
    if (await _isEmulator()) {
      risks.add(SecurityRisk.emulator);
    }
    
    // Check 3: Debug mode
    if (await _isDebugMode()) {
      risks.add(SecurityRisk.debugMode);
    }
    
    // Check 4: Tampering detection
    if (await _isAppTampered()) {
      risks.add(SecurityRisk.tampered);
    }
    
    // Check 5: VPN/Proxy detection
    if (await _isProxyActive()) {
      risks.add(SecurityRisk.proxyActive);
    }
    
    return SecurityStatus(
      isSecure: risks.isEmpty,
      risks: risks,
      timestamp: DateTime.now(),
    );
  }
  
  static Future<bool> _isDeviceRooted() async {
    try {
      // Check for root files (Android)
      if (Platform.isAndroid) {
        final result = await _channel.invokeMethod<bool>('checkRoot');
        return result ?? false;
      }
      
      // Check for jailbreak (iOS)
      if (Platform.isIOS) {
        final result = await _channel.invokeMethod<bool>('checkJailbreak');
        return result ?? false;
      }
      
      return false;
    } catch (e) {
      // If check fails, assume compromised
      return true;
    }
  }
  
  static Future<bool> _isEmulator() async {
    if (Platform.isAndroid) {
      final result = await _channel.invokeMethod<bool>('checkEmulator');
      return result ?? false;
    }
    return false;
  }
  
  static Future<bool> _isDebugMode() async {
    return bool.fromEnvironment('dart.vm.product') == false;
  }
  
  static Future<bool> _isAppTampered() async {
    // Check signature/certificate
    final result = await _channel.invokeMethod<bool>('verifySignature');
    return result == false;
  }
  
  static Future<bool> _isProxyActive() async {
    // Check proxy settings
    return Platform.environment.containsKey('HTTP_PROXY') ||
           Platform.environment.containsKey('http_proxy');
  }
}

class SecurityStatus {
  final bool isSecure;
  final List<SecurityRisk> risks;
  final DateTime timestamp;
  
  SecurityStatus({
    required this.isSecure,
    required this.risks,
    required this.timestamp,
  });
  
  bool get shouldBlockAccess => risks.contains(SecurityRisk.rootedDevice) || 
                                 risks.contains(SecurityRisk.tampered);
}

enum SecurityRisk {
  rootedDevice,
  emulator,
  debugMode,
  tampered,
  proxyActive,
}
```

**Explanation:**

- **Root Detection**: Rooted/jailbroken devices bypass OS security controls. Malware can access app sandbox data.
- **Emulator Detection**: Emulators often used for dynamic analysis. Block or restrict functionality.
- **Tamper Detection**: Verify app signature hasn't changed (re-signing attacks).
- **Proxy Detection**: MITM attacks often use proxy servers. Detect and warn user.
- **Defense Strategy**: Don't just block; implement "graceful degradation" (disable sensitive features, increase monitoring, warn user).

### **Native Root Detection (Android)**

```kotlin
// android/src/main/kotlin/com/example/app/RootChecker.kt
package com.example.app

import android.content.Context
import android.os.Build
import java.io.File

object RootChecker {
    
    fun isDeviceRooted(): Boolean {
        return checkTestKeys() || 
               checkSuperUserApk() || 
               checkSuBinary() ||
               checkBusyBox() ||
               checkMagisk()
    }
    
    // Check for test-keys in build tags
    private fun checkTestKeys(): Boolean {
        val buildTags = android.os.Build.TAGS
        return buildTags != null && buildTags.contains("test-keys")
    }
    
    // Check for Superuser.apk
    private fun checkSuperUserApk(): Boolean {
        return File("/system/app/Superuser.apk").exists()
    }
    
    // Check for su binary in common locations
    private fun checkSuBinary(): Boolean {
        val paths = arrayOf(
            "/system/bin/su",
            "/system/xbin/su",
            "/sbin/su",
            "/su/bin/su",
            "/data/local/xbin/su",
            "/data/local/bin/su",
            "/system/sd/xbin/su",
            "/system/bin/failsafe/su",
            "/data/local/su"
        )
        return paths.any { File(it).exists() }
    }
    
    // Check for BusyBox (common on rooted devices)
    private fun checkBusyBox(): Boolean {
        return try {
            Runtime.getRuntime().exec("which busybox").waitFor() == 0
        } catch (e: Exception) {
            false
        }
    }
    
    // Check for Magisk (modern root solution)
    private fun checkMagisk(): Boolean {
        return File("/sbin/.magisk").exists() ||
               File("/data/adb/magisk").exists()
    }
    
    // SafetyNet/Play Integrity API check
    fun checkPlayIntegrity(context: Context): Boolean {
        // Implementation would call Google Play Integrity API
        // Returns attestation of device integrity
        return true // Placeholder
    }
}
```

**Explanation:**

- **Multiple Checks**: Root detection is cat-and-mouse. Combine multiple checks (file existence, build tags, API calls).
- **Test-Keys**: Official Android builds have "release-keys". "test-keys" indicate custom ROM/root.
- **su Binary**: The root privilege escalation binary. Check common paths but note that Magisk can hide this.
- **Play Integrity API**: Google's official attestation API (replaces SafetyNet). Verifies device integrity with Google servers.
- **Limitations**: Determined attackers can bypass root detection (Magisk Hide, etc.). Treat as "best effort" not absolute security.

---

## **42.4 OWASP Mobile Top 10 Mitigation**

Implementing controls for the most critical mobile security risks.

### **M2: Insecure Data Storage**

```dart
// lib/security/data_protection.dart
import 'dart:convert';
import 'dart:typed_data';
import 'package:encrypt/encrypt.dart' as encrypt;

/// Service for encrypting sensitive data before storage
class DataEncryptionService {
  static encrypt.Key? _encryptionKey;
  
  /// Initialize with device-specific key from Keystore/Keychain
  static Future<void> initialize() async {
    // Retrieve or generate key from secure hardware
    final keyString = await SecureStorageService.read(key: 'data_encryption_key');
    
    if (keyString == null) {
      // Generate new AES-256 key
      final key = encrypt.Key.fromSecureRandom(32);
      await SecureStorageService.write(
        key: 'data_encryption_key', 
        value: key.base64,
      );
      _encryptionKey = key;
    } else {
      _encryptionKey = encrypt.Key.fromBase64(keyString);
    }
  }
  
  /// Encrypt sensitive data for local storage
  static String encryptData(String plaintext) {
    if (_encryptionKey == null) {
      throw StateError('Encryption service not initialized');
    }
    
    final iv = encrypt.IV.fromSecureRandom(16);
    final encrypter = encrypt.Encrypter(
      encrypt.AES(_encryptionKey!, mode: encrypt.AESMode.gcm),
    );
    
    final encrypted = encrypter.encrypt(plaintext, iv: iv);
    
    // Combine IV + ciphertext for storage
    return '${iv.base64}:${encrypted.base64}';
  }
  
  /// Decrypt data from local storage
  static String decryptData(String ciphertext) {
    if (_encryptionKey == null) {
      throw StateError('Encryption service not initialized');
    }
    
    final parts = ciphertext.split(':');
    if (parts.length != 2) {
      throw FormatException('Invalid encrypted format');
    }
    
    final iv = encrypt.IV.fromBase64(parts[0]);
    final encrypted = encrypt.Encrypted.fromBase64(parts[1]);
    
    final encrypter = encrypt.Encrypter(
      encrypt.AES(_encryptionKey!, mode: encrypt.AESMode.gcm),
    );
    
    return encrypter.decrypt(encrypted, iv: iv);
  }
}

/// Secure cache implementation
class SecureCache {
  final Map<String, String> _memoryCache = {};
  
  Future<void> put(String key, String value) async {
    // Encrypt before storing in memory (protects from memory dump)
    final encrypted = DataEncryptionService.encryptData(value);
    _memoryCache[key] = encrypted;
  }
  
  Future<String?> get(String key) async {
    final encrypted = _memoryCache[key];
    if (encrypted == null) return null;
    return DataEncryptionService.decryptData(encrypted);
  }
  
  void clear() {
    _memoryCache.clear();
    // Force garbage collection to clear memory
  }
}
```

**Explanation:**

- **AES-256-GCM**: Encrypts sensitive data before writing to SharedPreferences or files. GCM provides authentication (detects tampering).
- **IV Management**: Each encryption uses unique IV (Initialization Vector). Stored alongside ciphertext.
- **Memory Encryption**: Encrypts data even in RAM. Protects from memory dump attacks and process inspection.
- **Key from Keystore**: Encryption key stored in Android Keystore/iOS Keychain, not in app code.

### **M5: Insufficient Cryptography**

```dart
// lib/security/crypto_utils.dart
import 'dart:convert';
import 'package:crypto/crypto.dart';
import 'package:pointycastle/export.dart';

class SecureCrypto {
  /// Generate cryptographically secure random token
  static String generateSecureToken(int length) {
    final secureRandom = SecureRandom('Fortuna');
    final keyParams = KeyParameter(Uint8List(32));
    secureRandom.seed(keyParams);
    
    final bytes = secureRandom.nextBytes(length);
    return base64UrlEncode(bytes);
  }
  
  /// Hash password using PBKDF2
  static String hashPassword(String password, String salt) {
    final pbkdf2 = PBKDF2KeyDerivator(HMac(SHA256Digest(), 64));
    final params = Pbkdf2Parameters(
      utf8.encode(salt) as Uint8List, 
      100000, // Iterations (high for security)
      32,      // Key length
    );
    
    pbkdf2.init(params);
    final bytes = pbkdf2.process(utf8.encode(password) as Uint8List);
    return base64Encode(bytes);
  }
  
  /// Constant-time comparison to prevent timing attacks
  static bool constantTimeCompare(String a, String b) {
    if (a.length != b.length) return false;
    
    var result = 0;
    final aBytes = utf8.encode(a);
    final bBytes = utf8.encode(b);
    
    for (var i = 0; i < aBytes.length; i++) {
      result |= aBytes[i] ^ bBytes[i];
    }
    
    return result == 0;
  }
  
  /// NEVER use for passwords - only for data integrity
  static String sha256Hash(String input) {
    return sha256.convert(utf8.encode(input)).toString();
  }
}
```

**Explanation:**

- **SecureRandom**: Uses Fortuna CSPRNG (Cryptographically Secure Pseudo-Random Number Generator). Never use `Random()` for security tokens.
- **PBKDF2**: Password-Based Key Derivation Function. Slow by design (100k iterations) to resist brute force.
- **Constant-Time Comparison**: Prevents timing attacks that can leak hash information through response time differences.
- **SHA-256**: Fast hash, **never** use for passwords. Use only for data integrity checks.

---

## **Chapter Summary**

In this chapter, we covered comprehensive security implementation for Flutter applications:

### **Key Takeaways:**

1. **Secure Storage**: Use platform-specific secure storage (Android Keystore, iOS Keychain) via method channels. Never store keys in code or SharedPreferences/UserDefaults. Use AES-256-GCM for data encryption.

2. **Certificate Pinning**: Pin certificates in Dart (`SecurityContext`) and native network security config (Android). Validate certificate fingerprints to prevent MITM attacks. Include backup pins for rotation.

3. **Code Obfuscation**: Enable Dart obfuscation with `--obfuscate` flag. Configure ProGuard/R8 rules for Android. Store symbol files securely for crash reporting.

4. **Root/Jailbreak Detection**: Implement multiple detection methods (file checks, build tags, API attestation). Use Play Integrity API for official attestation. Implement graceful degradation rather than hard blocking.

5. **OWASP Mitigation**:
   - **M2**: Encrypt sensitive data at rest with hardware-backed keys
   - **M5**: Use strong cryptography (AES-256, SHA-256, PBKDF2), avoid deprecated algorithms (MD5, SHA1)
   - **M3**: Use secure network protocols (TLS 1.2+), certificate pinning, no cleartext traffic
   - **M9**: Protect against reverse engineering with obfuscation and anti-tampering

6. **Runtime Protection**: Implement RASP (Runtime Application Self-Protection) checks for debuggers, emulators, and tampering. Encrypt sensitive data even in memory.

### **Security Checklist:**
- ✅ API keys and secrets stored in native secure storage, not Dart code
- ✅ Network traffic uses HTTPS with certificate pinning
- ✅ Production builds use code obfuscation (`--obfuscate`)
- ✅ Android Network Security Config disables cleartext traffic and debug overrides
- ✅ Root detection implemented with graceful degradation
- ✅ Cryptographic keys use hardware-backed storage (TEE/Secure Enclave)
- ✅ Sensitive data encrypted in memory, not just at rest
- ✅ ProGuard/R8 rules preserve necessary classes while obfuscating app logic

---

## **Next Steps**

In the next chapter, **Chapter 43: Build & Release**, we will explore the complete build and release process for Flutter applications. You'll learn how to configure signing certificates for Android (AAB) and iOS (App Store), automate CI/CD pipelines using GitHub Actions and Codemagic, implement over-the-air updates, and manage beta testing through TestFlight and Google Play Console.

---

**End of Chapter 42**